<a href="https://colab.research.google.com/github/bill7845/colab_repo/blob/master/dacon_%EC%A0%9C%EC%A3%BC%EC%B9%B4%EB%93%9C%EC%82%AC%EC%9A%A9%EB%9F%89%EC%98%88%EC%B8%A1/modeling_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
data = pd.read_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/data/201901-202003.csv', encoding='utf-8')
submission_april = pd.read_csv("/content/drive/My Drive/Colab code/dacon_카드사용액 예측/data/202004.csv")
data.shape

(24697792, 12)

In [3]:
data = pd.concat([data,submission_april])
data

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3
...,...,...,...,...,...,...,...,...,...,...,...,...
1350317,202004,충북,충주시,휴양콘도 운영업,충북,충주시,20s,1,1,5,77000,5
1350318,202004,충북,충주시,휴양콘도 운영업,충북,충주시,30s,1,2,6,92000,6
1350319,202004,충북,충주시,휴양콘도 운영업,충북,충주시,40s,2,3,5,193000,5
1350320,202004,충북,충주시,휴양콘도 운영업,충북,충주시,50s,1,4,5,86000,7


In [4]:
## data preprocessing

data = data.fillna('세종') # 결측값 ''로 대체

## 날짜 처리
def grap_year(data): # year 
    data = str(data)
    return int(data[:4])

def grap_month(data): # month 
    data = str(data)
    return int(data[4:])

data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1) # 기존 날짜컬럼 삭제
data.head()

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,year,month
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,2019,1
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,2019,1
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,2019,1
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,2019,1
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,2019,1


In [5]:
# 결측치 확인
data.isnull().sum()

CARD_SIDO_NM    0
CARD_CCG_NM     0
STD_CLSS_NM     0
HOM_SIDO_NM     0
HOM_CCG_NM      0
AGE             0
SEX_CTGO_CD     0
FLC             0
CSTMR_CNT       0
AMT             0
CNT             0
year            0
month           0
dtype: int64

In [6]:
# 시군구도 필요없으니 시군구 변수도 삭제
del data['CARD_CCG_NM']
del data['HOM_CCG_NM']
del data['CSTMR_CNT']
del data['CNT']

In [7]:
## SEX_CTGE_CD 인코딩
sex_condi = {1:0, 2:1} # 남자 0 여자 1로 인코딩
data['SEX_CTGO_CD'] = data['SEX_CTGO_CD'].map(sex_condi)

# ## 지역민인지 여부 변수 추가해보기
# def get_local_nlocal(df):
#   if df['CARD_SIDO_NM'] == df['HOM_SIDO_NM']:
#     return 1 # 지역민 1
#   else :
#     return 0 # 외지인 0

# data['local_nlocal'] = data.apply(get_local_nlocal,axis=1)

# 컬럼 순서
data = data[['year','month','CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM','AGE','SEX_CTGO_CD','FLC','AMT']] # local_nlocal

* 지역별 모델링

In [42]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from itertools import product
import lightgbm as lgb

## encoding def
## 인코딩 완료된 df형태로 반환
def encoder_func(df):
  dtypes = df.dtypes
  encoders = {}
  for column in df.columns:
    if str(dtypes[column]) == 'object':
      encoder = LabelEncoder()
      encoder.fit(df[column])
      encoders[column] = encoder

  for column in encoders.keys():
    encoder = encoders[column]
    df[column] = encoder.transform(df[column])

  return df,encoders


# 지역별 모델링
def modeling(full_data): # full_data는 전체 데이터의미. (data)

  res_model_dict = {} # 각 지역별 모델 저장할 딕셔너리
  res_forecasting = pd.DataFrame(columns=['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','AMT']) # res_forecating에 각 지역별 예측결과를 누적하여 저장.
  rmse_df = pd.DataFrame(columns=['best rmse'])
  city_Nonexistent_std_clss = {}

  city_list = full_data['CARD_SIDO_NM'].unique() # 전체 지역 리스트
  
  for city in city_list : #city_list
    print(city) 
    sampling = full_data.query("CARD_SIDO_NM == @city") # 전체 데이터에서 한 지역씩 sampling 변수에 담음
    Nonexistent_std_clss = list( set(full_data['STD_CLSS_NM'].unique()) - set(sampling['STD_CLSS_NM'].unique())) # 각 지역별로 집계값이 0인 업종들
    city_Nonexistent_std_clss[city] = Nonexistent_std_clss
    sampling,encoders = encoder_func(sampling) # 위에서 정의한 인코딩함수에 sampling을 넣어 인코딩 된 데이터와 인코더 객체를 반환받음

    train_num = sampling.sample(frac=1, random_state=0) # 데이터 셔플
    train_features = train_num.drop(['CARD_SIDO_NM','AMT'],axis=1) # features # 각 지역별로 진행하기 때문에 CARD_SIDO_NM은 제외함.
    train_target = np.log1p(train_num['AMT']) # target

    k = int(len(train_features)*0.9)
    
    x_train = train_features[:k]
    y_train = train_target[:k]
    x_val = train_features[k:]
    y_val = train_target[k:]


    params = {
                'learning_rate' : 0.05,
                'boosting_type': 'gbdt',
                'objective': 'tweedie',
                'tweedie_variance_power': 1.1,
                'metric': 'rmse',
                'sub_row' : 0.75,
                'lambda_l2' : 0.1
            }
        
    train_ds = lgb.Dataset(x_train, label=y_train)
    val_ds = lgb.Dataset(x_val, label=y_val)    
    # evals_result = {}
    model = lgb.train(params,
                  train_ds,
                  1000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 100
                  # evals_result = evals_result
                 )
    
    # res_model_dict['model_'+city] = model # 학습 된 모델을 res_model 딕셔너리에 저장

    # rmse_df.loc[city] = [min(min(evals_result['valid_0'].values()))]

    ## 템플릿 생성 부분
    ## 각 지역별로 진행하기 때문에 CARD_SIDO_NM은 제외함.
    year_temp = [2020]
    month_temp = [7]
    STD_CLSS_NM_temp = sampling['STD_CLSS_NM'].unique()
    HOM_SIDO_NM_temp = sampling['HOM_SIDO_NM'].unique()
    AGE_temp = sampling['AGE'].unique()
    SEX_CTGO_CD_temp = sampling['SEX_CTGO_CD'].unique()
    FLC_temp = sampling['FLC'].unique()
    comb_list = [year_temp, month_temp, STD_CLSS_NM_temp, HOM_SIDO_NM_temp, AGE_temp, SEX_CTGO_CD_temp, FLC_temp] 
    forecasting_temp = np.array(list(product(*comb_list)))
    forecasting_temp = pd.DataFrame(data=forecasting_temp, columns=train_features.columns) # 예측 템플릿 완성

    pred = model.predict(forecasting_temp) # 예측
    pred = np.expm1(pred)
    forecasting_temp['AMT'] = np.round(pred, 0) 
    forecasting_temp['REG_YYMM'] = forecasting_temp['year']*100 + forecasting_temp['month']
    forecasting_temp = forecasting_temp[['REG_YYMM','STD_CLSS_NM', 'AMT']]
    forecasting_temp.insert(1,'CARD_SIDO_NM',city) # 첫번째 열에 "CARD_SIDO_NM"이라는 열을 추가함 값은 해당 지역이름임
    forecasting_temp = forecasting_temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False) # 지역별,업종별 월간 사용금액 합계
    forecasting_temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(forecasting_temp['STD_CLSS_NM']) # STD_CLSS_NM inverse encoding
    
    ## 각 지역별로 집계값이 0인 업종들이 있음
    ## submission에서는 이 업종들도 포함해서 제출해야 하기 때문에 0으로 때려넣을것임
    for clss in Nonexistent_std_clss:
      tmp = pd.DataFrame([[ 202007, city, clss, 0 ]],columns=forecasting_temp.columns)
      forecasting_temp = pd.concat([forecasting_temp,tmp]) # forecasting_temp에 concat
    
    res_forecasting = pd.concat([res_forecasting,forecasting_temp]) # res_forecasting에 지역별 예측완료 된 값들을 누적하여 저장
    
  return res_model_dict, res_forecasting, city_Nonexistent_std_clss # 지역별 모델 딕셔너리, 최종 예측값 반환, 지역별 best rmse , 지역,업종별 null인 업종 # rmse_df

In [44]:
########################## 모델 실행 부분 ########################################

# res_model_dict_full,res_forecasting_full,city_Nonexistent_std_clss = modeling(data) # 실행 # 전체 기간 반영 # rmse_df_full

data_2020 = data.query("year==2020 and (month==3 or month==4)") # 2020년 2/3월 데이터로만
res_model_dict_2020,res_forecasting_2020,city_Nonexistent_std_clss_s = modeling(data_2020) # 실행 # 2020년 2,3월만 반영 # rmse_df_2020


########################## 모델 결과물을 최종 제출물 형태로  ########################################

# submisson과 최종 예측 프레임 합치기
def get_submission(data):
  submission = pd.read_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/data/submission.csv', index_col=0)
  submission = submission.drop(['AMT'], axis=1)
  submission = submission.merge(data, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
  submission.index.name = 'id'

  return submission

submission_2020 = get_submission(res_forecasting_2020)
submission_2020
# submission_full = get_submission(res_forecasting_full)

강원
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 1.2275
[200]	valid_0's rmse: 1.2183
[300]	valid_0's rmse: 1.21688
[400]	valid_0's rmse: 1.21653
Early stopping, best iteration is:
[362]	valid_0's rmse: 1.21641
경기
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 1.49036
[200]	valid_0's rmse: 1.47898
[300]	valid_0's rmse: 1.47558
[400]	valid_0's rmse: 1.47364
[500]	valid_0's rmse: 1.47258
[600]	valid_0's rmse: 1.47183
[700]	valid_0's rmse: 1.47142
[800]	valid_0's rmse: 1.47105
[900]	valid_0's rmse: 1.4707
[1000]	valid_0's rmse: 1.4704
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 1.4704
경남
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 1.52292
[200]	valid_0's rmse: 1.51383
[300]	valid_0's rmse: 1.51163
[400]	valid_0's rmse: 1.51092
[500]	valid_0's rmse: 1.51073
Early stopping, best iteration is:
[486]	valid_0's rmse: 1.51069
경북
Training until validatio

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,NaN
1,202004,강원,골프장 운영업,NaN
2,202004,강원,과실 및 채소 소매업,NaN
3,202004,강원,관광 민예품 및 선물용품 소매업,NaN
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,NaN
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,49065390.0
1390,202007,충북,한식 음식점업,146164480.0
1391,202007,충북,호텔업,98992886.0


In [47]:
# submission_2020.to_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/submission/기혁_final.csv', encoding='utf-8-sig') # 저장 경로 설정 후 실행

In [ ]:
# 공개된 4월 데이터


########################## 4월 데이터도 동일한 전처리 과정  ########################################

submission_april = pd.read_csv("/content/drive/My Drive/Colab code/dacon_카드사용액 예측/data/202004.csv")

submission_april = submission_april.fillna('세종') # 결측값 ''로 대체

submission_april['year'] = submission_april['REG_YYMM'].apply(lambda x: grap_year(x))
submission_april['month'] = submission_april['REG_YYMM'].apply(lambda x: grap_month(x))
submission_april = submission_april.drop(['REG_YYMM'], axis=1) # 기존 날짜컬럼 삭제
submission_april.head()

# 시군구도 필요없으니 시군구 변수도 삭제
del submission_april['CARD_CCG_NM']
del submission_april['HOM_CCG_NM']
del submission_april['CSTMR_CNT']
del submission_april['CNT']

## SEX_CTGE_CD 인코딩
sex_condi = {1:0, 2:1} # 남자 0 여자 1로 인코딩
submission_april['SEX_CTGO_CD'] = submission_april['SEX_CTGO_CD'].map(sex_condi)

# 컬럼 순서
submission_april = submission_april[['year','month','CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM','AGE','SEX_CTGO_CD','FLC','AMT']] # local_nlocal

submission_april['REG_YYMM'] = submission_april['year']*100 + submission_april['month']
submission_april = submission_april[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
submission_april = submission_april.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)


########################## 4월 데이터도 동일한 전처리 과정  ########################################
"""
공개된 4월데이터에 기존에 없던 null값(카드사용금액이 미미한 경우) 발생. 이 부분 0으로 때려넣음
"""

std_clss_list = data['STD_CLSS_NM'].unique() # 모든 업종 unique값

def plus_std_clss(data, std_clss_list): # 4월데이터의 null값을 0으로 채워넣은 후 반환

  result_df = pd.DataFrame(columns=['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','AMT'])
  plus_Nonexistent_std_clss = {}

  for city in data['CARD_SIDO_NM'].unique():
    sampling = data.query("CARD_SIDO_NM==@city")
    Nonexistent_std_clss = list( set(std_clss_list) - set(sampling['STD_CLSS_NM'].unique()) )
    plus_Nonexistent_std_clss[city] = Nonexistent_std_clss
    
    for clss in Nonexistent_std_clss:
      tmp=pd.DataFrame([[ 202004, city, clss, 0 ]], columns=sampling.columns)
      sampling = pd.concat([sampling,tmp])
    
    result_df = pd.concat([result_df,sampling])
  
  return result_df,plus_Nonexistent_std_clss

submission_april,plus_Nonexistent_std_clss = plus_std_clss(submission_april, std_clss_list) # 함수 실행

## original submission과 index 맞추는 과정
tmp = pd.read_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/data/submission.csv', index_col=0)
tmp = tmp[tmp['REG_YYMM'] == 202004] # 4월만 추출
submission_april = tmp.merge(submission_april, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission_april.index.name = 'id'
del submission_april['AMT_x']
submission_april = submission_april.rename(columns={"AMT_y" : "AMT"})
submission_april

In [ ]:
######## 4월 데이터에 null값으로 추가된 업종 ## 지역별로
res_dict = {}
for city in city_Nonexistent_std_clss:
  res_dict[city] = list( set(plus_Nonexistent_std_clss[city]) - set(city_Nonexistent_std_clss[city]) )

res_dict

In [ ]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error

actual = submission_april['AMT'].values

predict_1 = submission_2020.query("REG_YYMM==202004")['AMT'].values
res_1 = np.sqrt(mean_squared_log_error(actual, predict_1))
print('1 :',res_1)

##############################################

predict_2 = submission_full.query("REG_YYMM==202004")['AMT'].values
res_2 = np.sqrt(mean_squared_log_error(actual, predict_2))
print('2 :',res_2)

In [ ]:
######################################################################

In [ ]:
data_t = pd.read_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/data/201901-202003.csv', encoding='utf-8')
data_t = data_t.fillna('')

df = data_t.copy()
df = df[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
df = df.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)
df = df.loc[df['REG_YYMM']==202003]
df = df[['CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]

submission_t = pd.read_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/data/submission.csv', index_col=0)
submission_t = submission_t.loc[submission_t['REG_YYMM']==202004]
submission_t = submission_t[['CARD_SIDO_NM', 'STD_CLSS_NM']]
submission_t = submission_t.merge(df, left_on=['CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission_t = submission_t.fillna(0)
AMT = list(submission_t['AMT'])*2

submission_t = pd.read_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/data/submission.csv', index_col=0)
submission_t['AMT'] = AMT
# submission_t.to_csv('submission.csv', encoding='utf-8-sig')
submission_t.head()

In [ ]:
np.sqrt(mean_squared_log_error(submission_april['AMT'].values, submission_t.query("REG_YYMM==202004")['AMT'].values))

In [ ]:
data_a = pd.read_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/data/201901-202003.csv', encoding='utf-8')
data_a = data_a.fillna('세종')

df = data_a.copy()
df = df[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
df = df.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)
df = df.loc[df['REG_YYMM']==202003]
df = df[['CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]

submission_a = pd.read_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/data/submission.csv', index_col=0)
submission_a = submission_a.loc[submission_a['REG_YYMM']==202004]
submission_a = submission_a[['CARD_SIDO_NM', 'STD_CLSS_NM']]
submission_a = submission_a.merge(df, left_on=['CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission_a = submission_a.fillna(0)
AMT = list(submission_a['AMT'])*2

submission_a = pd.read_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/data/submission.csv', index_col=0)
submission_a['AMT'] = AMT
# submission_t.to_csv('submission.csv', encoding='utf-8-sig')
submission_a.head()

In [ ]:
np.sqrt(mean_squared_log_error(submission_april['AMT'].values, submission_a.query("REG_YYMM==202004")['AMT'].values))